Introduction to Advanced Node and Express Challenges
Authentication is the process or action of verifying the identity of a user or process. Up to this point you have not been able to create an app utilizing this key concept.

The most common and easiest way to use authentication middleware for Node.js is Passport. It is easy to learn, light-weight, and extremely flexible allowing for many strategies, which we will talk about in later challenges. In addition to authentication we will also look at template engines which allow for use of Pug and web sockets which allow for real time communication between all your clients and your server. 

## Set up a Template Engine
A template engine enables you to use static template files (such as those written in Pug) in your app. At runtime, the template engine replaces variables in a template file with actual values which can be supplied by your server, and transforms the template into a static HTML file that is then sent to the client. This approach makes it easier to design an HTML page and allows for displaying of variables on the page without needing to make an API call from the client.

To set up Pug for use in your project, you will need to add it as a dependency first in your package.json. "pug": "^0.1.0"

Now to tell Node/Express to use the templating engine you will have to tell your express app to set 'pug' as the 'view-engine'. app.set('view engine', 'pug')

Lastly, you should change your response to the request for the index route to res.render with the path to the view views/pug/index.pug.

<details><summary>code</summary>
app.set('view engine', 'pug');<br />

app.route('/')<br />
  .get((req, res) => {<br />
    res.render(process.cwd() + '/views/pug/index.pug'); <br />
  });
</details>

## Use a Template Engine's Powers
As a reminder, this project is being built upon the following starter project on Glitch, or cloned from GitHub.

One of the greatest features of using a template engine is being able to pass variables from the server to the template file before rendering it to HTML.

In your Pug file, you're able to use a variable by referencing the variable name as #{variable_name} inline with other text on an element or by using an equal sign on the element without a space such as p=variable_name which assigns the variable's value to the p element's text.

We strongly recommend looking at the syntax and structure of Pug here on GitHub's README. Pug is all about using whitespace and tabs to show nested elements and cutting down on the amount of code needed to make a beautiful site.

Looking at our pug file 'index.pug' included in your project, we used the variables title and message.

To pass those along from our server, you will need to add an object as a second argument to your res.render with the variables and their values. For example, pass this object along setting the variables for your index view: {title: 'Hello', message: 'Please login'}

It should look like: 
```javascript
res.render(process.cwd() + '/views/pug/index', {title: 'Hello', message: 'Please login'});
```

## Set up Passport
In addition to Passport, we will use Express-session to handle sessions. Using this middleware saves the session id as a cookie in the client and allows us to access the session data using that id on the server. This way we keep personal account information out of the cookie used by the client to verify to our server they are authenticated and just keep the key to access the data stored on the server.

To set up Passport for use in your project, you will need to add it as a dependency first in your package.json. "passport": "^0.3.2"

In addition, add Express-session as a dependency now as well. Express-session has a ton of advanced features you can use but for now we're just going to use the basics! "express-session": "^1.15.0"

You will need to set up the session settings now and initialize Passport. Be sure to first create the variables 'session' and 'passport' to require 'express-session' and 'passport' respectively.

To set up your express app to use the session we'll define just a few basic options. Be sure to add 'SESSION_SECRET' to your .env file and give it a random value. This is used to compute the hash used to encrypt your cookie!
```javascript
app.use(session({
  secret: process.env.SESSION_SECRET,
  resave: true,
  saveUninitialized: true,
}));
```
As well you can go ahead and tell your express app to use 'passport.initialize()' and 'passport.session()'. (For example, app.use(passport.initialize());) 

## Serialization of a User Object
As a reminder, this project is being built upon the following starter project on Glitch, or cloned from GitHub.

Serialization and deserialization are important concepts in regards to authentication. To serialize an object means to convert its contents into a small key essentially that can then be deserialized into the original object. This is what allows us to know whos communicated with the server without having to send the authentication data like username and password at each request for a new page.

To set this up properly, we need to have a serialize function and a deserialize function. In passport we create these with passport.serializeUser( OURFUNCTION ) and passport.deserializeUser( OURFUNCTION )

The serializeUser is called with 2 arguments, the full user object and a callback used by passport. Returned in the callback should be a unique key to identify that user- the easiest one to use being the users _id in the object as it should be unique as it generated by MongoDb. Similarly deserializeUser is called with that key and a callback function for passport as well, but this time we have to take that key and return the users full object to the callback. To make a query search for a Mongo _id you will have to create const ObjectID = require('mongodb').ObjectID;, and then to use it you call new ObjectID(THE_ID). Be sure to add MongoDB as a dependency. You can see this in the examples below:
```javascript
passport.serializeUser((user, done) => {
  done(null, user._id);
});
passport.deserializeUser((id, done) => {
  db.collection('users').findOne(
    {_id: new ObjectID(id)},
      (err, doc) => {
        done(null, doc);
      }
  );
});
```
NOTE: This deserializeUser will throw an error until we set up the DB in the next step so comment out the whole block and just call done(null, null) in the function deserializeUser. Submit your page when you think you've got it right.

## Serialization of a User Object
As a reminder, this project is being built upon the following starter project on Glitch, or cloned from GitHub.

Serialization and deserialization are important concepts in regards to authentication. To serialize an object means to convert its contents into a small key essentially that can then be deserialized into the original object. This is what allows us to know whos communicated with the server without having to send the authentication data like username and password at each request for a new page.

To set this up properly, we need to have a serialize function and a deserialize function. In passport we create these with passport.serializeUser( OURFUNCTION ) and passport.deserializeUser( OURFUNCTION )

The serializeUser is called with 2 arguments, the full user object and a callback used by passport. Returned in the callback should be a unique key to identify that user- the easiest one to use being the users _id in the object as it should be unique as it generated by MongoDb. Similarly deserializeUser is called with that key and a callback function for passport as well, but this time we have to take that key and return the users full object to the callback. To make a query search for a Mongo _id you will have to create const ObjectID = require('mongodb').ObjectID;, and then to use it you call new ObjectID(THE_ID). Be sure to add MongoDB as a dependency. You can see this in the examples below:
```javascript
passport.serializeUser((user, done) => {
  done(null, user._id);
});
passport.deserializeUser((id, done) => {
  db.collection('users').findOne(
    {_id: new ObjectID(id)},
      (err, doc) => {
        done(null, doc);
      }
  );
});
```

## Implement the Serialization of a Passport User
Right now we're not loading an actual user object since we haven't set up our database. This can be done many different ways, but for our project we will connect to the database once when we start the server and keep a persistent connection for the full life-cycle of the app.

To do this, add MongoDB as a dependency and require it in your server. (const mongo = require('mongodb').MongoClient;)

Now we want to the connect to our database then start listening for requests. The purpose of this is to not allow requests before our database is connected or if there is a database error. To accomplish you will want to encompass your serialization and your app listener in the following:
```javascript
mongo.connect(process.env.DATABASE, (err, db) => {
  if(err) {
    console.log('Database error: ' + err);
  } else {
    console.log('Successful database connection');

    //serialization and app.listen
  }
});
```
<details><summary>code</summary>
mongo.connect(process.env.DATABASE, (err, db) => { <br />
  if(err) {<br />
    console.error("Database error: "+ err)<br />
  } else {<br />
  console.log('Successful database connection');<br />
  <br />
  passport.serializeUser((user, done)=> {<br />
    done(null, user._id)<br />
  });<br />
  <br />
  passport.deserializeUser((id, done)=> {<br />
    db.collection('users').findOne({_id: new ObjectID(id)}, <br />
                                   (err, doc) =>{<br />
      done(null, doc)<br />
    })<br />
  })<br />
  app.route('/')<br />
  .get((req, res) => {<br />
    res.render(process.cwd() + '/views/pug/index.pug', {title: 'Hello', message: "Please login"});<br />
  });<br />

<br />

<br />
app.listen(process.env.PORT || 3000, () => {<br />
  console.log("Listening on port " + process.env.PORT);<br />
});<br />
    <br />
  }})<br />
</details>

## Authentication Strategies
A strategy is a way of authenticating a user. You can use a strategy for allowing users to authenticate based on locally saved information (if you have them register first) or from a variety of providers such as Google or GitHub. For this project we will set up a local strategy. To see a list of the 100's of strategies, visit Passports site here.

Add passport-local as a dependency and add it to your server as follows: const LocalStrategy = require('passport-local');

Now you will have to tell passport to use an instantiated LocalStartegy object with a few settings defined. Make sure this as well as everything from this point on is encapsulated in the database connection since it relies on it!
```javascript
passport.use(new LocalStrategy(
  function(username, password, done) {
    db.collection('users').findOne({ username: username }, function (err, user) {
      console.log('User '+ username +' attempted to log in.');
      if (err) { return done(err); }
      if (!user) { return done(null, false); }
      if (password !== user.password) { return done(null, false); }
      return done(null, user);
    });
  }
));
```

## How to Use Passport Strategies
In the index.pug file supplied there is actually a login form. It has previously been hidden because of the inline JavaScript if showLogin with the form indented after it. Before showLogin as a variable was never defined, it never rendered the code block containing the form. Go ahead and on the res.render for that page add a new variable to the object showLogin: true. When you refresh your page, you should then see the form! This form is set up to POST on /login so this is where we should set up to accept the POST and authenticate the user.

For this challenge you should add the route /login to accept a POST request. To authenticate on this route you need to add a middleware to do so before then sending a response. This is done by just passing another argument with the middleware before your function(req,res) with your response! The middleware to use is passport.authenticate('local').

passport.authenticate can also take some options as an argument such as: { failureRedirect: '/' } which is incredibly useful so be sure to add that in as well. As a response after using the middleware (which will only be called if the authentication middleware passes) should be to redirect the user to /profile and that route should render the view 'profile.pug'.

If the authentication was successful, the user object will be saved in req.user.

Now at this point if you enter a username and password in the form, it should redirect to the home page / and in the console of your server should be 'User {USERNAME} attempted to log in.' since we currently cannot login a user who isn't registered.
<details><summary>code</summary>

 app.route('/login').post(passport.authenticate('local', {failureRedirect: '/'}),(req, res) => { <br />
      res.redirect('/profile')<br />
    })<br />
<br />

app.route('/profile').get((req, res) => {<br />
  res.render(process.cwd()+ '/views/pug/profile.pug');<br />
})<br />
</details>

## Create New Middleware
As in, any user can just go to /profile whether they authenticated or not by typing in the url. We want to prevent this by checking if the user is authenticated first before rendering the profile page. This is the perfect example of when to create a middleware.

The challenge here is creating the middleware function ensureAuthenticated(req, res, next), which will check if a user is authenticated by calling passports isAuthenticated on the request which in turn checks for req.user is to be defined. If it is then next() should be called, otherwise we can just respond to the request with a redirect to our homepage to login. An implementation of this middleware is:
```javascript
function ensureAuthenticated(req, res, next) {
  if (req.isAuthenticated()) {
    return next();
  }
  res.redirect('/');
};
```
Now add ensureAuthenticated as a middleware to the request for the profile page before the argument to the get request containing the function that renders the page.
```javascript
app
 .route('/profile')
 .get(ensureAuthenticated, (req,res) => {
    res.render(process.cwd() + '/views/pug/profile');
 });
 ```
 
## Logging a User Out
Creating the logout logic is easy. The route should just unauthenticate the user and redirect to the home page instead of rendering any view.

In passport, unauthenticating a user is as easy as just calling req.logout(); before redirecting.
```javascript
app.route('/logout')
  .get((req, res) => {
    req.logout();
    res.redirect('/');
});
```
You may have noticed that we're not handling missing pages (404), the common way to handle this in Node is with the following middleware. Go ahead and add this in after all your other routes:
```javascript
app.use((req, res, next) => {
  res.status(404)
    .type('text')
    .send('Not Found');
});
```
